# <img src="https://img.icons8.com/fluency/96/000000/particles.png" style="height:50px;display:inline"> Deep Latent Particles - Tutorial
---

Tal Daniel



<center>
    <a href="https://colab.research.google.com/github/taldatech/deep-latent-particles-pytorch/blob/main/dlp_tutorial.ipynb"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>
</center>

* Paper: [**Unsupervised Image Representation Learning with Deep Latent Particles**, Tal Daniel and Aviv Tamar, ICML 2022](https://arxiv.org/)
* GitHub: <a href="https://github.com/taldatech/deep-latent-particles-pytorch">deep-latent-particles-pytorch</a>
* Wepage: <a href="https://taldatech.github.io/deep-latent-particles-web/">ICML 2022 - Deep Latent Particles</a>

<img src="https://raw.githubusercontent.com/taldatech/deep-latent-particles-web/main/assets/clevrer_manip_1.gif" style="height:250px">

### <img src="https://img.icons8.com/color/96/000000/loading.png" style="height:50px;display:inline"> Running Instructions
---
* This Jupyter Notebook can be opened locally with Anaconda, or online via Google Colab.
* To run online, go to https://colab.research.google.com/ and drag-and-drop the `dlp_tutorial.ipynb` file.
    * On Colab, note the "directory" icon on the left, logs, figures and checkpoints are saved in this directory.
* To run the training on the image dataset, it is better to have a GPU. In Google Cola select `Runtime->Change runtime type->GPU`.

### <img src="https://img.icons8.com/color/96/000000/software-installer.png" style="height:50px;display:inline"> Requirements
---
* If running locally, make sure to set up the environment, preferably with <a href="https://anaconda.org/">Anaconda</a> by running `conda env create -f environment17/19.yml` or with `pip` by running `pip install - r requirements17/19.txt`.
* If running online on <a href="https://colab.research.google.com">Google Colab</a>, run the following cell to clone the repository.

In [1]:
# only run this cell if running on Google Colab
!git clone https://github.com/taldatech/deep-latent-particles-pytorch.git
!cd deep-latent-particles-pytorch

Cloning into 'deep-latent-particles-pytorch'...
remote: Enumerating objects: 155, done.
remote: Counting objects: 100% (155/155), done.
remote: Compressing objects: 100% (106/106), done.
remote: Total 155 (delta 67), reused 127 (delta 43), pack-reused 0
Receiving objects: 100% (155/155), 18.21 MiB | 9.09 MiB/s, done.
Resolving deltas: 100% (67/67), done.


### <img src="https://img.icons8.com/bubbles/50/000000/checklist.png" style="height:50px;display:inline"> Agenda
---
* [Variational Autoencoders (VAEs)](#-Variational-Autoencoders-(VAEs))
* [Soft-IntroVAE Objectives](#-Soft-IntroVAE-Objectives)
* [Image Generation Experiments](#-Image-Generation-Experiments)
    * [Image Generation Experiments - Architectures](#Image-Generation-Experiments---Architectures)
    * [Image Generation Experiments - Algorithm and Train Function](#Image-Generation-Experiments---Algorithm-and-Train-Function)
* [Credits](#-Credits)

### <img src="https://img.icons8.com/bubbles/50/000000/mention.png" style="height:50px;display:inline"> Introduction
---
* In this tutorial, we will give a brief overview of the deep latent particles (DLP) framework. 
* DLP is a new representation of visual data that disentangles object position from appearance. 
* DLP decomposes the visual input into low-dimensional latent ``particles'', where each particle is described by its spatial location and features of its surrounding region. 
* DLP is trained with a variational autoencoder (VAE) objective with a special prior for particle positions based on a spatial-softmax architecture, and a modification of the evidence lower bound loss inspired by the Chamfer distance between particles.

## <img src="https://img.icons8.com/dusk/64/000000/prize.png" style="height:50px;display:inline"> Credits
---
* Icons from <a href="https://icons8.com/">Icon8.com</a> - https://icons8.com